In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm

In [ ]:
lens_diameter = 60. # lens diameter in mm

class Optics(tlm.Module):
    def __init__(self):
        super().__init__()
        
        self.shape1 = tlm.CircularArc(lens_diameter, nn.Parameter(torch.tensor(-200.)))
        self.shape2 = tlm.Line(lens_diameter, [0.0, 1.0, 0.0])# y=0 line

        self.optics = nn.Sequential(
            tlm.ParallelBeamRandom(0.9*lens_diameter),
            tlm.GapY(15.),
            
            tlm.RefractiveSurface(self.shape1, (1.0, 1.49), anchors=("origin", "extent")),
            tlm.GapY(5.),
            tlm.RefractiveSurface(self.shape2, (1.49, 1.0)),
            
            tlm.GapY(100.0), # focal length
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)

optics = Optics()

init = (10, torch.tensor([0., 0.]))
tlm.render_plt(optics, init)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    inputs = init,
    num_iter = 150
)

tlm.render_plt(optics, init)

In [ ]:
lens = tlm.Lens(optics.stack[2], optics.stack[3], optics.stack[4])

part = tlm.lens_to_part(lens)
part